In [9]:
import pandas as pd
import sqlite3

In [10]:
sqlite3.connect('../Server/db.sqlite3').cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('django_migrations',),
 ('sqlite_sequence',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('django_admin_log',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_user',),
 ('django_session',),
 ('Logs_log',),
 ('Areas_area',),
 ('Devices_device',),
 ('Locations_location',)]

In [11]:
areas = pd.read_sql_query('SELECT * FROM Areas_area', sqlite3.connect('../Server/db.sqlite3'))
areas

,id,value,label
0,1,1,LAMCE


In [12]:
devices = pd.read_sql_query('SELECT * FROM Devices_device', sqlite3.connect('../Server/db.sqlite3'))
devices

,variables,id,esp_id,timestamp,area,master,node
0,"[""temperature"", ""humidity""]",1,08AB29DB511,2025-07-21 10:44:01.824829,1,0,pier_grva
1,"[""pressure"", ""humidity""]",2,B80DB629E74,2025-09-03 18:37:41.927310,1,1,grva


In [13]:
def getInfo(id):
    target = devices.loc[devices.esp_id == id]

    if len(target) == 0:
        return None

    node   = devices.iloc[0].node
    area   = devices.iloc[0].area
    target = areas.loc[areas.value == area]
    
    if len(areas) == 0:
        return None
    
    label = target.iloc[0].label
    return {'area': int(area), 'label': label, 'node': node}


getInfo('08AB29DB511')

{'area': 1, 'label': 'LAMCE', 'node': 'pier_grva'}

In [14]:
locations = pd.read_sql_query('SELECT * FROM Locations_location', sqlite3.connect('../Server/db.sqlite3'))
locations

,id,label,value,lat,lng
0,1,Pier LAMCE,pier_grva,-22.862688,-43.225895


In [29]:
locations['device'] = None

for i, loc in locations.iterrows():
    for j, dev in devices.iterrows():
        if loc.value != dev.node:
            continue

        locations.at[i, 'device'] = dev.esp_id 

locations

,id,label,value,lat,lng,device
0,1,Pier LAMCE,pier_grva,-22.862688,-43.225895,08AB29DB511


In [33]:
[row.to_dict() for i, row in locations.iterrows()]

[{'id': 1,
  'label': 'Pier LAMCE',
  'value': 'pier_grva',
  'lat': -22.862688,
  'lng': -43.225895,
  'device': '08AB29DB511'}]